# Non-Gaussian Measurements Tutorial 2

## Overview

- This tutorial introduces non-Gaussian estimation via a factor graph solution.
- Factor graphs should be thought of fully probabilisitic interactions between variables, as modelled by the factors.
- This tutorial shows how measurement statistics do not have be unimodal bell curve (i.e. normal/Gaussian) and thereby introduce non-Gaussian behavior.
- This example will use only continuous stochastic variables with slightly more diverse measurement distribution.
- This tutorial is one dimension with only a few variables and factors, in order to get comfortable working with multi-modal beliefs.
- This example is showing one of four mechanisms by which non-Gaussian behavior can get introduced into a factor graph solution, see other tutorials for other mechanisms.
- This tutorial illustrates how algebraic relations (i.e. residual functions) between multiple stochastic variables are calculated, as well as the final posterior belief estimate, from several pieces of information.
- The tutorial implicitly shows a multi-modal uncertainty can be introduced from non-Gaussian measurements, and then transmitted through the system.
- The tutorial also illustrates consensus through an additional piece of information, which reduces all stochastic variable marginal beliefs to unimodal only beliefs.
- The ambiguous measurement data example shown in this tutorial can readily be incorporated in other use cases.
- Lastly, this tutorial will also show how graph-based automatic initialization of variables is achieved with the Caesar.jl solver.

### Signatures Used

`ContinuousScalar`, `Prior`, `LinearRelative`, `Mixture`, `Normal`, `plotKDE`, `plotDFG`, `isInitialized`, `initAll!`, `solveGraph!`

## Ambiguous Data Example

The application of this tutorial is presented in abstract from which the user is free to imagine any system of relationships:  For example, a robot driving in a one dimensional world; or a time traveler making uncertain jumps 
forwards and backwards in time.

To start, the two major mathematical packages are brought into scope.

In [2]:
from navability.entities import *
from navability.services import *
from uuid import uuid4
import asyncio

 
## Starting a 1D Factor Graph

### Variable, `ContinuousScalar`

The next step is to describe the inference problem with a graphical model with any of the existing concrete types that inherit from  `<: AbstractDFG`.
The first step is to create an empty factor graph object and start populating it with variable nodes.
The variable nodes are identified by `Symbol`s, namely `:x0, :x1, :x2, :x3`.


In [3]:
# Start with an empty factor graph
navability_client = NavAbilityHttpsClient()
client = Client("Guest", "PySDKAutomation", "Session_" + str(uuid4())[0:8])
print(client)

<Client(userId=Guest, robotId=PySDKAutomation, sessionId=Session_998eb539)>


In [4]:
# add the first node
v = Variable("x0", "IncrementalInference.ContinuousScalar") #VariableType.ContinuousScalar.value)
result_id = await addVariable(navability_client, client, v)
print(f"Added {v.label} with result ID {result_id}")


Added x0 with result ID 01FYYDWQ9T2WHRBC4CS6MKWSXP


In [5]:
# Confirm it's there
print(await getStatusLatest(navability_client, result_id))
# TODO:  This will fail: https://github.com/NavAbility/NavAbilitySDK.py/issues/40

<StatusMessage(requestId=01FYYDWQ9T2WHRBC4CS6MKWSXP, timestamp=2022-03-24 16:59:27.754000, client=<Client(userId=Guest, robotId=PySDKAutomation, sessionId=Session_998eb539)>, action=Add:Variable, state=Complete)>


### Prior Factor, (Euclidean(1))

Factor graphs are bipartite graphs with `factors` that act as mathematical structure between interacting `variables`.
After adding node `:x0`, a singleton factor of type `Prior` (which was defined by the user earlier) is 'connected to' variable node `:x0`.
This unary factor is taken as a `Distributions.Normal` distribution with zero mean and a standard devitation of `1`.


In [6]:
# this is unary (prior) factor and does not immediately trigger autoinit of :x0.
# TODO: https://github.com/NavAbility/NavAbilitySDK.py/issues/43
prior = Factor("x0f1", "Prior", ["x0"],
      FactorData(
          fnc=Prior(
              Z=Normal(0, 1)
          ).dump()  # This is a generator for a Prior
      ))
# REF: addFactor!(fg, [:x0], Prior(Normal(0,1)))
result_id = await addFactor(navability_client, client, prior)

In [8]:
# Checking
print(await getStatusLatest(navability_client, result_id))

<StatusMessage(requestId=01FYYDX4J1PGKHFEA9PWGVCH9M, timestamp=2022-03-24 16:59:42.609000, client=<Client(userId=Guest, robotId=PySDKAutomation, sessionId=Session_998eb539)>, action=Add:Factor, state=Complete)>


**Note**, this example is using just `Prior`, which is a simplification for the sake of this tutorial.  Look out for on-manifold prior factors in other code examples.

### Visualizing Graph Topology

In [ ]:
# TODO: Create a shortlink to visualization of the graph.
# TODO: https://github.com/NavAbility/NavAbilitySDK.py/issues/44
plotDFG(fg)


The two node factor graph is shown in the image below.
```@raw html
<p align="center">
<img src="https://raw.githubusercontent.com/JuliaRobotics/Caesar.jl/master/docs/src/assets/tutorials/ContinuousScalar/fgx0.png" width="120" border="0" />
</p>
```

### Graph-based Variable Initialization

Automatic initialization of variables depend on how the factor graph model is constructed.
This tutorial demonstrates this behavior by first showing that `:x0` is not initialized:

In [ ]:
# We don't do this in SDK.
#@show isInitialized(fg, :x0) # false

Why is `:x0` not initialized?
Since no other variable nodes have been 'connected to' (or depend) on `:x0` and future intentions of the user are unknown, the initialization of `:x0` is deferred until the latest possible moment.
`IncrementalInference.jl` assumes that the user will generally populate new variable nodes with most of the associated factors before moving to the next variable.
By delaying initialization of a new variable (say `:x0`) until a second newer uninitialized variable (say `:x1`) depends on `:x0`, the `IncrementalInference` algorithms hope to then initialize `:x0` with the more information from previous and surrounding variables and factors.
Also note that graph-based initialization of variables is a local operation based only on the neighboring nodes -- global inference occurs over the entire graph and is shown later in this tutorial.


## Factor `LinearRelative`

By adding `:x1` and connecting it through the `LinearRelative` and `Normal` distributed factor, the automatic initialization of `:x0` is triggered.

In [9]:
# REF: addVariable!(fg, :x1, ContinuousScalar)
v = Variable("x1", "IncrementalInference.ContinuousScalar") #VariableType.ContinuousScalar.value)
result_id = await addVariable(navability_client, client, v)
print(f"Added {v.label} with result ID {result_id}")

Added x1 with result ID 01FYYDYEXPG3H4QAHK5TDA3WCY


In [10]:
# Checking
print(await getStatusLatest(navability_client, result_id))

<StatusMessage(requestId=01FYYDYEXPG3H4QAHK5TDA3WCY, timestamp=2022-03-24 17:00:22.060000, client=<Client(userId=Guest, robotId=PySDKAutomation, sessionId=Session_998eb539)>, action=Add:Variable, state=Complete)>


In [11]:
# P(Z | :x1 - :x0 ) where Z ~ Normal(10,1)
# REF: addFactor!(fg, [:x0, :x1], LinearRelative(Normal(10.0,1)))
linear_factor = Factor("x0x1f1", "LinearRelative", ["x0", "x1"],
      FactorData(
          fnc=LinearRelative(Normal(10, 0.1)
          ).dump()  # This is a generator for a LinearRelative
      ))
# REF: addFactor!(fg, [:x0], Prior(Normal(0,1)))
result_id = await addFactor(navability_client, client, linear_factor)
#@show isInitialized(fg, :x0) # true

In [13]:
# Checking
print(await getStatusLatest(navability_client, result_id))

<StatusMessage(requestId=01FYYDYTBPE4YKQE7HBDDXYAW2, timestamp=2022-03-24 17:00:37.427000, client=<Client(userId=Guest, robotId=PySDKAutomation, sessionId=Session_998eb539)>, action=Add:Factor, state=Complete)>



Note that the automatic initialization of `:x0` is aware that `:x1` is not initialized and therefore only used the `Prior(Normal(0,1))` unary factor to initialize the marginal belief estimate for `:x0`.
The structure of the graph has now been updated to two variable nodes and two factors.
```@raw html
<p align="center">
<img src="https://raw.githubusercontent.com/JuliaRobotics/Caesar.jl/master/docs/src/assets/tutorials/ContinuousScalar/fgx01.png" width="240" border="0" />
</p>
```

Notice how the previous `:x0` is now initialized,

In [ ]:
#@show isInitialized(fg, :x0)

and the new `:x1` variable is not yet initialized as described before:

In [ ]:
#@show isInitialized(fg, :x1)

### Visualizing the Variable Probability Belief

The `RoMEPlotting.jl` package allows visualization (plotting) of the belief state over any of the variable nodes.
Remember the first time executions are slow given required code compilation, and that future versions of these package will use more precompilation to reduce first execution running cost.

In [ ]:
# TODO: https://github.com/NavAbility/nva-sdk/issues/19
#using RoMEPlotting

#plotKDE(fg, :x0)


```@raw html
<p align="center">
<img src="https://raw.githubusercontent.com/JuliaRobotics/Caesar.jl/master/docs/src/assets/tutorials/ContinuousScalar/plx0.png" width="360" border="0" />
</p>
```
By forcing the initialization of `:x1` and plotting its belief estimate,

In [ ]:
#initAll!(fg)
#plotKDE(fg, [:x0, :x1])

the predicted influence of the `P(Z| X1 - X0) = LinearRelative(Normal(10, 1))` is shown by the red trace.
```@raw html
<p align="center">
<img src="https://raw.githubusercontent.com/JuliaRobotics/Caesar.jl/master/docs/src/assets/tutorials/ContinuousScalar/plx01.png" width="360" border="0" />
</p>
```
The red trace (predicted belief of `:x1`) is noting more than the approximated convolution of the current marginal belief of `:x0` with the conditional belief described by `P(Z | X1 - X0)`.


### Mixture Distribution on Next Relative Factor

Another `ContinuousScalar` variable `:x2` is 'connected' to `:x1` through a more complicated `MixtureRelative` likelihood function.

In [14]:
# REF: addVariable!(fg, :x2, ContinuousScalar)
# mmo = Mixture(LinearRelative, 
#              (hypo1=Rayleigh(3), hypo2=Uniform(30,55)), 
#              [0.4; 0.6])
# addFactor!(fg, [:x1, :x2], mmo)
v = Variable("x2", "IncrementalInference.ContinuousScalar") #VariableType.ContinuousScalar.value)
result_id = await addVariable(navability_client, client, v)
print(f"Added {v.label} with result ID {result_id}")


Added x2 with result ID 01FYYE0CZARNGB6WKEPQSVFV9K


In [15]:
print(await getStatusLatest(navability_client, result_id))

<StatusMessage(requestId=01FYYE0CZARNGB6WKEPQSVFV9K, timestamp=2022-03-24 17:01:25.654000, client=<Client(userId=Guest, robotId=PySDKAutomation, sessionId=Session_998eb539)>, action=Add:Variable, state=Complete)>


In [18]:
mixture_factor = Factor("x1x2f1", "Mixture", ["x1", "x2"],
      FactorData(
            fnc=Mixture(LinearRelative, 
                {
                    "hypo1": Rayleigh(3), 
                    "hypo2": Uniform(30,55)
                },
                [0.4, 0.6],
                1
          ).dump()
      ))
print(mixture_factor.dumps())
# REF: addFactor!(fg, [:x0], Prior(Normal(0,1)))
result_id = await addFactor(navability_client, client, mixture_factor)

{"label": "x1x2f1", "_version": "0.18.1", "_variableOrderSymbols": ["x1", "x2"], "data": {"eliminated": false, "potentialused": false, "edgeIDs": [], "fnc": {"N": 2, "F_": "PackedLinearRelative", "S": ["hypo1", "hypo2"], "components": [{"_type": "IncrementalInference.PackedRayleigh", "sigma": 3.0}, {"_type": "IncrementalInference.PackedUniform", "a": 30.0, "b": 55.0, "PackedSamplableTypeJSON": "IncrementalInference.PackedUniform"}], "diversity": {"_type": "IncrementalInference.PackedCategorical", "p": [0.4, 0.6]}}, "multihypo": [], "certainhypo": [1, 2], "nullhypo": 0.0, "solveInProgress": 0, "inflation": 5.0}, "tags": ["FACTOR"], "timestamp": "2022-03-24T16:59:01.927Z", "nstime": "0", "fnctype": "Mixture", "solvable": 1}


In [23]:
# Checking
print(await getStatusLatest(navability_client, result_id))

<StatusMessage(requestId=01FYYM477QE66QHCQ6NGEV1EE4, timestamp=2022-03-24 18:48:30.353000, client=<Client(userId=Guest, robotId=PySDKAutomation, sessionId=Session_998eb539)>, action=Add:Factor, state=Complete)>



```@raw html
<p align="center">
<img src="https://raw.githubusercontent.com/JuliaRobotics/Caesar.jl/master/docs/src/assets/tutorials/ContinuousScalar/fgx012.png" width="360" border="0" />
</p>
```
The `mmo` variable illustrates how a near arbitrary mixture probability distribution can be used as a conditional relationship between variable nodes in the factor graph.
In this case, a 40%/60% balance of a Rayleigh and truncated Uniform distribution which acts as a multi-modal conditional belief.
Interpret carefully what a conditional belief of this nature actually means.

Following the tutorial's practical example frameworks (robot navigation or time travel), this multi-modal belief implies that moving from one of the probable locations in `:x1` to a location in `:x2` by some processes defined by `mmo=P(Z | X2, X1)` is uncertain to the same 40%/60% ratio.
In practical terms, collapsing (through observation of an event) the probabilistic likelihoods of the transition from `:x1` to `:x2` may result in the `:x2` location being at either 15-20, or 40-65-ish units.
The predicted belief over `:x2` is illustrated by plotting the predicted belief (green trace), after forcing initialization.

In [ ]:
#initAll!(fg)
#plotKDE(fg, [:x0, :x1, :x2])

```@raw html
<p align="center">
<img src="https://raw.githubusercontent.com/JuliaRobotics/Caesar.jl/master/docs/src/assets/tutorials/ContinuousScalar/plx012.png" width="360" border="0" />
</p>
```

## Adding Variable `x3`

Adding one more variable `:x3` through another `LinearRelative(Normal(-50,1))`

In [26]:
# REF: addVariable!(fg, :x3, ContinuousScalar)
# REF: addFactor!(fg, [:x2, :x3], LinearRelative(Normal(-50, 1)))
v = Variable("x3", "IncrementalInference.ContinuousScalar") #VariableType.ContinuousScalar.value)
result_id = await addVariable(navability_client, client, v)
print(f"Added {v.label} with result ID {result_id}")

linear_factor = Factor("x2x3f1", "LinearRelative", ["x2", "x3"],
      FactorData(
          fnc=LinearRelative(Normal(-50, 1)
          ).dump()  # This is a generator for a LinearRelative
      ))
result_id = await addFactor(navability_client, client, linear_factor)

Added x3 with result ID 01FYVCCKHSQ6RVWB616ZRN7ZM3


In [27]:
# Checking
print(await getStatusLatest(navability_client, result_id))

<StatusMessage(requestId=01FYVCCM36S5Z0XPB9WD0T7YMV, timestamp=2022-03-23 12:35:29.870000, client=<Client(userId=Guest, robotId=PySDKAutomation, sessionId=Session_0869d232)>, action=Add:Factor, state=Complete)>


expands the factor graph to to four variables and four factors.
```@raw html
<p align="center">
<img src="https://raw.githubusercontent.com/JuliaRobotics/Caesar.jl/master/docs/src/assets/tutorials/ContinuousScalar/fgx0123.png" width="480" border="0" />
</p>
```
This part of the tutorial shows how a unimodal likelihood (conditional belief) can transmit the bimodal belief currently contained in `:x2`.

In [ ]:
#initAll!(fg)
#plotKDE(fg, [:x0, :x1, :x2, :x3])

Notice the blue trace (`:x3`) is a shifted and slightly spread out version of the initialized belief on `:x2`, through the convolution with the conditional belief `P(Z | X2, X3)`.
```@raw html
<p align="center">
<img src="https://raw.githubusercontent.com/JuliaRobotics/Caesar.jl/master/docs/src/assets/tutorials/ContinuousScalar/plx0123.png" width="480" border="0" />
</p>
```

### The Last Factor


Global inference over the entire factor graph has still not occurred, and will at this stage produce roughly similar results to the predicted beliefs shown above.
Only by introducing more information into the factor graph can inference extract more precise marginal belief estimates for each of the variables.
A final piece of information added to this graph is a factor directly relating `:x3` with `:x0`.

In [28]:
# REF: addFactor!(fg, [:x3, :x0], LinearRelative(Normal(40, 1)))
linear_factor = Factor("x3x0f1", "LinearRelative", ["x3", "x0"],
      FactorData(
          fnc=LinearRelative(Normal(40, 1)
          ).dump()  # This is a generator for a LinearRelative
      ))
result_id = await addFactor(navability_client, client, linear_factor)

In [30]:
# Checking
print(await getStatusLatest(navability_client, result_id))

<StatusMessage(requestId=01FYVCDFRJE3SW8ARY17H2JHQ2, timestamp=2022-03-23 12:35:56.747000, client=<Client(userId=Guest, robotId=PySDKAutomation, sessionId=Session_0869d232)>, action=Add:Factor, state=Complete)>


Pay close attention to what this last factor means in terms of the probability density traces shown in the previous figure.
The blue trace for `:x3` has two major modes, one that overlaps with `:x0, :x1` near 0 and a second mode further to the left at -40.
The last factor introduces a shift `LinearRelative(Normal(40,1))` which essentially aligns the left most mode of `:x3` back onto `:x0`.
```@raw html
<p align="center">
<img src="https://raw.githubusercontent.com/JuliaRobotics/Caesar.jl/master/docs/src/assets/tutorials/ContinuousScalar/fgx0123c.png" width="480" border="0" />
</p>
```

This last factor forces a mode selection through consensus.
By doing global inference, the new information obtained in `:x3` will be equally propagated to `:x2` where only one of the two modes will remain.

## Solve the Graph


Global inference is achieved with local computation using two function calls, as follows.


In [25]:
#tree = solveGraph!(fg)

# and visualization
#plotKDE(fg, [:x0, :x1, :x2, :x3])
result_id = solveSession(navability_client, client)

In [ ]:
# Checking
print(await getStatusLatest(navability_client, result_id))

## Viewing the results

In [36]:
# Making a button using markdown
from IPython.display import Markdown as md
#See the results
topography_vis_link = f"https://app.navability.io/cloud/graph?userId={client.userId}&robotStartsWith={client.robotId}&sessionStartsWith={client.sessionId}"
spatial_vis_link = f"https://app.navability.io/cloud/map?userId={client.userId}&robotStartsWith={client.robotId}&sessionStartsWith={client.sessionId}"
md(f"""[![Navigate to Factor Graph](http://www.navability.io/wp-content/uploads/2022/03/factor_graph.png)]({topography_vis_link})
  [![Navigate to Geometic Map](http://www.navability.io/wp-content/uploads/2022/03/geometric_map.png)]({spatial_vis_link})""")



[![Navigate to Factor Graph](http://www.navability.io/wp-content/uploads/2022/03/factor_graph.png)](https://app.navability.io/cloud/graph?userId=Guest&robotStartsWith=PySDKAutomation&sessionStartsWith=Session_998eb539)
  [![Navigate to Geometic Map](http://www.navability.io/wp-content/uploads/2022/03/geometric_map.png)](https://app.navability.io/cloud/map?userId=Guest&robotStartsWith=PySDKAutomation&sessionStartsWith=Session_998eb539)

The resulting posterior marginal beliefs over all the system variables are:
```@raw html
<p align="center">
<img src="https://raw.githubusercontent.com/JuliaRobotics/Caesar.jl/master/docs/src/assets/tutorials/ContinuousScalar/plx0123infr.png" width="480" border="0" />
</p>
```

## Conclusion

It is import to note that although this tutorial ends with all marginal beliefs having near Gaussian shape and are unimodal, that the package supports multi-modal belief estimates during both the prediction and global inference processes.
In fact, many of the same underlying inference functions are involved with the automatic initialization process and the global multi-modal iSAM inference procedure.
This concludes the ContinuousScalar tutorial